In [5]:
import pandas as pd
import numpy as np
import re
import os

def clean_plant_name(col_name):
    """
    Extracts the plant name from the column headers.
    Examples:
    'Luas Panen Jahe (meter persegi) (M2)' -> 'Jahe'
    'Produksi Pisang (Ton)' -> 'Pisang'
    """
    # Remove common prefixes
    name = re.sub(r'^(Luas Panen|Produksi)\s+', '', col_name, flags=re.IGNORECASE)
    # Remove units in parentheses (e.g., (M2), (Kg), (pohon))
    name = re.sub(r'\s*\(.*?\)', '', name)
    return name.strip()

def load_and_prep(lp_path, prod_path, year):
    """
    Loads LP (Harvest Area) and Prod (Production) files for a given year,
    cleans them, and merges them into a single annual dataframe.
    """
    # Load data
    df_lp = pd.read_csv(lp_path)
    df_prod = pd.read_csv(prod_path)

    # --- FILTER 1: Drop 'Keji Beling' Columns ---
    df_lp = df_lp.loc[:, ~df_lp.columns.str.contains('Keji Beling', case=False)]
    df_prod = df_prod.loc[:, ~df_prod.columns.str.contains('Keji Beling', case=False)]

    # --- FILTER 2: Strict Province Cleaning ---
    def clean_prov_data(df):
        if 'Provinsi' not in df.columns:
            return df

        # 1. Drop rows where Provinsi is NaN
        df = df.dropna(subset=['Provinsi'])

        # 2. Convert to string and strip whitespace
        df['Provinsi'] = df['Provinsi'].astype(str).str.strip()

        # 3. Define invalid values (Exact matches to remove)
        invalid_exact = ['nan', '0', '', 'indonesia']

        # 4. Define text patterns to remove (Rows containing these words)
        # BPS files often have footers like "Angka Tetap", "Angka Sementara", "Sumber:", "Catatan:"
        blacklist_keywords = [
            'catatan',
            'angka tetap',
            'angka sementara',
            'angka sangat sementara',
            'sumber',
            'jumlah',
            'total'
        ]

        # Filter logic
        # Start with rows that are NOT in invalid_exact
        mask = ~df['Provinsi'].str.lower().isin(invalid_exact)

        # Loop through blacklist and remove any row containing these words
        for kw in blacklist_keywords:
            mask = mask & ~df['Provinsi'].str.contains(kw, case=False, na=False)

        # Also remove rows that are purely numbers (like row numbers '1', '2', etc.)
        mask = mask & ~df['Provinsi'].str.match(r'^\d+$')

        return df[mask]

    df_lp = clean_prov_data(df_lp)
    df_prod = clean_prov_data(df_prod)

    # Clean columns mapping
    # Keep 'Provinsi' as is, rename others using regex
    lp_cols = {c: clean_plant_name(c) for c in df_lp.columns if c != 'Provinsi'}
    prod_cols = {c: clean_plant_name(c) for c in df_prod.columns if c != 'Provinsi'}

    df_lp.rename(columns=lp_cols, inplace=True)
    df_prod.rename(columns=prod_cols, inplace=True)

    # Melt to long format
    lp_melt = df_lp.melt(id_vars=['Provinsi'], var_name='Jenis Tanaman', value_name='Luas Panen')
    prod_melt = df_prod.melt(id_vars=['Provinsi'], var_name='Jenis Tanaman', value_name='Produksi')

    # Clean non-numeric values
    def clean_numeric(x):
        try:
            return float(x)
        except:
            return 0.0

    lp_melt['Luas Panen'] = lp_melt['Luas Panen'].apply(clean_numeric)
    prod_melt['Produksi'] = prod_melt['Produksi'].apply(clean_numeric)

    # Merge
    df_annual = pd.merge(lp_melt, prod_melt, on=['Provinsi', 'Jenis Tanaman'], how='outer')
    df_annual['Year'] = year

    # Fill NaNs with 0 (Only affects numeric columns usually, but good for safety)
    df_annual.fillna(0, inplace=True)

    return df_annual

def expand_to_daily(df_annual, year):
    """
    Expands annual data into daily data with randomized distribution.
    Ensures sum of days equals annual total.
    """
    start_date = f'{year}-01-01'
    end_date = f'{year}-12-31'
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    days_count = len(dates)

    # Create a scaffold dataframe with every date for every row in annual data
    n_rows = len(df_annual)

    # Repeat indices
    repeated_indices = np.repeat(df_annual.index, days_count)

    # Create the daily dataframe
    df_daily = df_annual.loc[repeated_indices].copy()

    # Assign dates
    df_daily['Date'] = np.tile(dates, n_rows)

    # --- Randomization Logic ---
    np.random.seed(42 + year)
    noise_lp = np.random.uniform(0.2, 1.8, size=len(df_daily))
    noise_prod = np.random.uniform(0.2, 1.8, size=len(df_daily))

    df_daily['weight_lp'] = noise_lp
    df_daily['weight_prod'] = noise_prod

    # Calculate sum of weights per original row
    weight_sums = df_daily.groupby(df_daily.index)[['weight_lp', 'weight_prod']].transform('sum')

    # Calculate Final Daily Values
    df_daily['Luas Panen Daily'] = df_daily['Luas Panen'] * (df_daily['weight_lp'] / weight_sums['weight_lp'])
    df_daily['Produksi Daily'] = df_daily['Produksi'] * (df_daily['weight_prod'] / weight_sums['weight_prod'])

    # Cleanup
    final_cols = ['Date', 'Provinsi', 'Jenis Tanaman', 'Luas Panen Daily', 'Produksi Daily']
    df_daily = df_daily[final_cols]
    df_daily.rename(columns={'Luas Panen Daily': 'Luas Panen', 'Produksi Daily': 'Produksi'}, inplace=True)

    return df_daily

# --- Main Execution ---

# Determine the directory where this script is located
try:
    script_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    script_dir = os.getcwd()

# File Paths
files = {
    '2023': {
        'lp': os.path.join(script_dir, 'Luas Panen Tanaman Biofarmaka Menurut Provinsi dan Jenis Tanaman, 2023.csv'),
        'prod': os.path.join(script_dir, 'Produksi Tanaman Biofarmaka Menurut Provinsi dan Jenis Tanaman, 2023.csv')
    },
    '2024': {
        'lp': os.path.join(script_dir, 'Luas Panen Tanaman Biofarmaka Menurut Provinsi dan Jenis Tanaman , 2024.csv'),
        'prod': os.path.join(script_dir, 'Produksi Tanaman Biofarmaka Menurut Provinsi dan Jenis Tanaman , 2024.csv')
    }
}

print("Processing 2023 data...")
df_2023_annual = load_and_prep(files['2023']['lp'], files['2023']['prod'], 2023)
df_2023_daily = expand_to_daily(df_2023_annual, 2023)

print("Processing 2024 data...")
df_2024_annual = load_and_prep(files['2024']['lp'], files['2024']['prod'], 2024)
df_2024_daily = expand_to_daily(df_2024_annual, 2024)

# Combine
print("Combining and filtering dates...")
df_combined = pd.concat([df_2023_daily, df_2024_daily], axis=0)

# Filter for requested range: Nov 1 2023 to Dec 31 2024
start_filter = pd.Timestamp('2023-11-01')
end_filter = pd.Timestamp('2024-12-31')

df_final = df_combined[(df_combined['Date'] >= start_filter) & (df_combined['Date'] <= end_filter)]

# Sorting
df_final = df_final.sort_values(by=['Date', 'Provinsi', 'Jenis Tanaman'])

# Save to CSV
output_filename = os.path.join(script_dir, 'Biofarmaka_Daily_Disaggregated_Nov23_Dec24.csv')
df_final.to_csv(output_filename, index=False)

print(f"Success! Data saved to {output_filename}")
print(f"Unique Provinces: {df_final['Provinsi'].nunique()}")
print("Provinces found:", df_final['Provinsi'].unique())
print(f"Total Rows: {len(df_final)}")
print(df_final.head())

Processing 2023 data...
Processing 2024 data...
Combining and filtering dates...


/tmp/ipython-input-1020142287.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Provinsi'] = df['Provinsi'].astype(str).str.strip()
/tmp/ipython-input-1020142287.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Provinsi'] = df['Provinsi'].astype(str).str.strip()
/tmp/ipython-input-1020142287.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

Success! Data saved to /content/Biofarmaka_Daily_Disaggregated_Nov23_Dec24.csv
Unique Provinces: 38
Provinces found: ['Aceh' 'Bali' 'Banten' 'Bengkulu' 'DI Yogyakarta' 'DKI Jakarta'
 'Gorontalo' 'Jambi' 'Jawa Barat' 'Jawa Tengah' 'Jawa Timur'
 'Kalimantan Barat' 'Kalimantan Selatan' 'Kalimantan Tengah'
 'Kalimantan Timur' 'Kalimantan Utara' 'Kepulauan Bangka Belitung'
 'Kepulauan Riau' 'Lampung' 'Maluku' 'Maluku Utara' 'Nusa Tenggara Barat'
 'Nusa Tenggara Timur' 'Papua' 'Papua Barat' 'Papua Barat Daya'
 'Papua Pegunungan' 'Papua Selatan' 'Papua Tengah' 'Riau' 'Sulawesi Barat'
 'Sulawesi Selatan' 'Sulawesi Tengah' 'Sulawesi Tenggara' 'Sulawesi Utara'
 'Sumatera Barat' 'Sumatera Selatan' 'Sumatera Utara']
Total Rows: 243390
        Date Provinsi Jenis Tanaman   Luas Panen      Produksi
0 2023-11-01     Aceh          Jahe  3796.813669  11123.005344
1 2023-11-01     Aceh   Jeruk Nipis    51.659819    239.176186
2 2023-11-01     Aceh      Kapulaga     7.418483     11.579233
3 2023-11-01   

In [4]:
print("hello")

hello
